### 봉림 데이터 분석

데이터의 경우 자동 복합 가공기 PLC에서 수신되는 데이터이며, 초 단위로 수집되는 데이터이다.

해당 기계의 경우, 1호기 왼팔 오른팔, 2호기 왼팔 오른팔 별로 절삭 구간 세팅값이 다르기에 왼팔과 오른팔을 분할하여 진행한다.

### Background Knowledge
1. 보링과 핀의 절삭 구간(5개 구간)은 서로 공유되는 구조이다.
    - 즉, 아래에서 진행할 라벨링의 경우 보링 및 핀의 이동 구간이다.
    - 다만 깎는 넓이가 다르다 보니까 회전 속도의 경우 차이가 날 수 있음.
2. 보링과 핀의 절삭 구간 중, 대기 지점은 90mm이며, 절삭 완료 후 후진 시 해당 지점으로 이동한다.
    - 90mm 아래로 이동하는 경우는 잘 없으며, 혹여 그런 값이 찍힌다면 해당 경우는 가공기 정비 시 발생할 수 있다고 함.
    - 5개 구간을 이동하기에 절삭 속도는 각각 다르지만, 후진의 경우 한 번에 후진하기 때문에 음수값은 5개가 찍히진 않는다.
3. 보링, 핀, 드릴의 경우 세 개의 드릴이 동시에 절삭함
4. 드릴의 경우 세 개의 구간에 걸쳐 절삭함
    - 대기위치, 절삭위치, 관통위치의 세 구간이 존재함
5. 작업이 아직 완료된 부분은 아니지만, 향후 17시가 되어 작업이 종료되면 그 날의 카운트 값은 초기화 할 예정.

### 목표
현재 봉림 금속의 경우 불량률을 낮추기 위해 드릴 팁을 더 쓸 수 있음에도 불구하고, 200개 가량 절삭이 완료되면 드릴 팁을 교체해버린다고 한다.

데이터 분석 및 AI 모델을 활용하여 드릴을 좀 더 오래 활용할 수 있는 방안을 모색해야 한다.

### 개발되어야 하는 모델
- 팁 교체 주기 예측 모델
- 시뮬레이션 모델(검사기 데이터 기반)

In [ ]:
import sys
sys.path.insert(0, r"C:\Users\signlab_039\Desktop\projects\bonglim\main")

import warnings
import pandas as pd
from collections import Counter
from MainProcess.data_prep.src.functions import *
from MainProcess.data_prep.src.preprocessing import *
# get_tip_count_dataset

warnings.filterwarnings("ignore")

In [ ]:
rename_cols = db_table_to_csv("CONT_COMP_FAC_DATA_ONE", including_index=False).columns

one_df = pd.read_csv("./data/1호기.csv")

one_df.columns = rename_cols
one_df['등록일'] = pd.to_datetime(one_df['등록일'])

In [ ]:
one_df['day_name'] = one_df['등록일'].dt.day_name()
one_df = one_df.loc[(one_df['day_name'] != "Sunday") & (one_df['day_name'] != "Saturday")].drop("day_name", axis=1)

In [ ]:
# load 1, 2 machine dataset
tip_count = pd.read_csv("./data/팁카운트.csv")
one_left, one_right = split_left_right(one_df, True, True)

In [ ]:
one_left = set_label_with_boring_location(one_left, [90, 222, 259, 288, 294, 302])
one_right = set_label_with_boring_location(one_right, [90, 226, 265, 295, 301, 306])

In [ ]:
# 1, 2호기 팁 카운트 분할
count_tip_one = tip_count.loc[tip_count['FTC_FAC_NO'] == 1].reset_index(drop=True)
count_tip_two = tip_count.loc[tip_count['FTC_FAC_NO'] == 2].reset_index(drop=True)

In [ ]:
# set day range on count tip dataset
one_left = one_left.loc[one_left['등록일'] >= count_tip_one['FTC_DATETIME'].min()].reset_index(drop=True)
one_right = one_right.loc[one_right['등록일'] >= count_tip_one['FTC_DATETIME'].min()].reset_index(drop=True)

In [ ]:
# 1호기 데이터 병합을 위한 컬럼명 변경
one_left.rename(columns={col : col+"-왼팔" for col in one_left.columns if col != "등록일"}, inplace=True)
one_right.rename(columns={col : col+"-오른팔" for col in one_right.columns if col != "등록일"}, inplace=True)

In [ ]:
# 1호기 데이터셋 병합
one_df = pd.concat([one_left.drop("등록일", axis=1), one_right], axis=1)

In [ ]:
one_df.to_csv("./1호기.csv", index=False)

### 팁 카운트 데이터 확인
1호기 팁 카운트 데이터부터 확인 

In [ ]:
count_tip_one.info()  # 결측치는 없는 것으로 확인

In [ ]:
# 0값이 포함된 데이터들 확인
idxes = []
for idx in range(count_tip_one.shape[0]):
    if 0 in count_tip_one.iloc[idx, :].values:
        idxes.append(idx)

include_zero = count_tip_one.loc[idxes]

In [ ]:
include_zero

In [ ]:
# 사용량이 0으로 변하는 시점의 패턴을 보기 위해 반복문 작성
columns = "FTC_TIP1_L FTC_TIP2_L FTC_TIP3_L FTC_TIP1_R FTC_TIP2_R FTC_TIP3_R FTC_DATETIME".split(" ")
test_df = count_tip_one[columns]
test_df['FTC_DATETIME'] = pd.to_datetime(test_df['FTC_DATETIME'])
test_df = test_df.loc[("2025-03-12 00:00:00" > test_df['FTC_DATETIME']) | ("2025-03-12 23:59:59" < test_df['FTC_DATETIME'])]

for idx in range(test_df.shape[0]):
    if Counter(test_df.iloc[idx, :].values.tolist())[0] == 1:  # 배열안의 숫자들을 세고, 0의 개수가 1인 데이터만 가지고 왔을 때
        if test_df.iloc[idx, :].values.tolist().index(0) != 0:
            print(test_df.iloc[idx, :].values.tolist())

# test_df

0 하나 포함된 것 모두 -> 맨 처음 찍힌 시간들을 모두 들고와서 최소 시간대 -> 드릴 팁 교체 시작
0 여섯개 포함된 것 모두 -> 제일 나중에 찍힌 시간 -> 드릴 팁 교체 끝시점

In [ ]:
include_zero

위 코드 실행 결과, 출력되는 것이 아무것도 없는 점으로 미루어 봤을 때, 0으로 변하는 시점의 경우 항상 FTC_TIP1_L이 먼저 0으로 변하는 것으로 확인

In [ ]:
count_tip_one.describe().T.style.background_gradient()

In [ ]:
one_df.describe().T.style.background_gradient()

### 정리
- 드릴을 제외한 사용량의 경우 최대치가 101이며, 드릴의 경우 해당 값의 두 배인 202까지 값들이 분포되어 있다.
- 항상 FTC_TIP1_L 부터 초기화 버튼이 눌러지며, 해당 버튼을 눌렀을 땐 이미 불량이 발생했음을 의미한다.

### 고려사항
팁 교체 구간을 어떤 부분으로 볼 것인가?

1. 사용량 중 FTC_TIP1_L값이 0으로 변하는 시점부터 다시 카운트값이 올라가는 지점 전까지
2. 사용량 모두 0으로 변한 시점부터 다시 카운트값이 올라가는 지점 전까지

우선 첫 번째 상황으로 팁 교체 구간을 정의해보자.

In [ ]:
timelines = get_tip_changed_timeline(count_tip_one)

In [ ]:
timelines

질의사항

1. 팁 교체 시 평균적으로 걸리는 시간
2. 팁 교체 판단은 육안으로 진행한다고 했는데, 작업이 완료되는 링크마다 육안검사를 진행하는지
3. 세팅 부하율의 경우 30이 최대치인데, 작업 시에는 최대 25정도로 밖에 올라가지 않는다. 

In [ ]:
fig = go.Figure()
for col in one_df.columns:
    fig.add_trace(go.Scatter(
        x=one_df['등록일'], y=one_df[col], name=col
    ))

for time in timelines:
    fig.add_vrect(
        x0=time[0],
        x1=time[1],
        fillcolor="black",
        opacity=0.7,
        annotation_text="change drill tips"
    )
fig.update_layout(hovermode="x unified")
fig.show()

그래프 상으로는 식별이 불가한 수준

### 요약 통계량 비교

팁 교체 전후 특정 몇분 간의 데이터를 관측(한 제품당 절삭 시간의 경우 평균적으로 32초(32개 행)정도 소요됨)

절삭 완료 후 복합 가공 이후의 모든 프로세스를 거친 링크가 완료 로트에 적재될 때 마다 제품을 검사한다고 쳐도,

바깥에서의 프로세스가 있기 때문에 육안으로 불량을 판단하려면 시간이 좀 걸릴 것으로 예상,

팁 교체 시작 전후 30분, 1시간의 자료들의 요약 통계량을 비교

In [ ]:
before_change = []
after_change = []
for time in timelines:
    before_one_hour_tip_change = one_df.loc[(str(time[0] - pd.Timedelta(minutes=30)) <= one_df['등록일']) & (one_df['등록일'] < str(time[0]))]
    after_one_hour_tip_change = one_df.loc[(str(time[1]) < one_df['등록일']) & (one_df['등록일'] <= str(time[1] + pd.Timedelta(minutes=30)))]
    
    before_change.append(before_one_hour_tip_change)
    after_change.append(after_one_hour_tip_change)

In [ ]:
describe_target = [col for col in one_df.columns if "부하율" in col]
before_change[4].describe().T.style.background_gradient()

In [ ]:
after_change[4].describe().T.style.background_gradient()

In [ ]:
set_one = pd.read_csv("./data/1호기 세팅값.csv")
set_two = pd.read_csv("./data/2호기 세팅값.csv")

set_one

In [ ]:
set_one.columns = db_table_to_csv("CONT_COMP_FAC_SET_ONE", including_index=False).columns

In [ ]:
recently_setted = set_one.iloc[-3:-1, :]

In [ ]:
recently_setted

In [ ]:
recently_setted.loc[recently_setted['위치']==0][['회전부하율%', '핀부하율%', '드릴 상,하 부하율']]

In [ ]:
recently_setted.loc[recently_setted['위치']==1][['회전부하율%', '핀부하율%', '드릴 상,하 부하율']]

In [ ]:
left_loads = recently_setted.loc[recently_setted['위치']==0][['회전부하율%', '핀부하율%', '드릴 상,하 부하율']].apply(lambda x: int(x/10))
right_loads = recently_setted.loc[recently_setted['위치']==1][['회전부하율%', '핀부하율%', '드릴 상,하 부하율']].apply(lambda x: int(x/10))

In [ ]:
# target = [col for col in one_df.columns if "부하율" in col]
# for col in target:
#     one_df[f"{col} MA"] = one_df[col].rolling(window=300).sum()

In [ ]:
# for col in one_df.columns:
#     load_variation = [0]
#     if "부하율" in col:
#         for idx in range(1, one_df.shape[0]):
#             before_value = one_df.iloc[idx-1, :][col]
#             now = one_df.iloc[idx, :][col]
            
#             variation = ((now - before_value) / before_value) * 100
#             load_variation.append(variation)
#         one_df[f"{col} 변화량"] = load_variation

In [ ]:
# # 토크값 추출 시도 (정확한 토크값 X, 근사치)
# one_df['left_bor_torque'] = (one_df['보링회전 부하율-왼팔'] / one_df['보링회전 RPM-왼팔']).round(4)
# one_df['left_pin_torque'] = (one_df['핀 회전 부하율-왼팔'] / one_df['핀 회전 RPM-왼팔']).round(4)
# one_df['right_bor_torque'] = (one_df['보링회전 부하율-오른팔'] / one_df['보링회전 RPM-오른팔']).round(4)
# one_df['right_pin_torque'] = (one_df['핀 회전 부하율-오른팔'] / one_df['핀 회전 RPM-오른팔']).round(4)

In [ ]:
# fig = go.Figure()
# for col in one_df.columns:
#     fig.add_trace(go.Scatter(
#         x=one_df['등록일'], y=one_df[col], name=col
#     ))
    
# for time in timelines:
#     fig.add_vrect(
#         x0=time[0],
#         x1=time[1],
#         annotation_text="drill tip changed",
#         fillcolor="green",
#         opacity=0.3
#     )
    
# fig.show()

In [ ]:
# # 새로운 Feature 생성
# one_df['rpm_to_feed_ratio_left'] = one_df['보링회전 RPM-왼팔'] / (one_df['보링 가공 전,후 속도-왼팔'] + 1e-6)  # 0으로 나누는 것을 방지
# one_df['rpm_load_interaction_left'] = one_df['보링회전 RPM-왼팔'] * one_df['보링회전 부하율-왼팔']
# one_df['feed_load_ratio_left'] = one_df['보링 가공 전,후 속도-왼팔'] / (one_df['보링회전 부하율-왼팔'] + 1e-6)
# one_df['rpm_feed_diff_left'] = one_df['보링회전 RPM-왼팔'] - one_df['보링 가공 전,후 속도-왼팔']
# one_df['load_rpm_feed_interaction_left'] = one_df['보링회전 부하율-왼팔'] * one_df['보링회전 RPM-왼팔'] * one_df['보링 가공 전,후 속도-왼팔']
# one_df['cutting_energy_left'] = one_df['보링회전 RPM-왼팔'] * one_df['보링 가공 전,후 속도-왼팔'] * one_df['보링회전 부하율-왼팔']

# # 오른팔도 동일하게 적용
# one_df['rpm_to_feed_ratio_right'] = one_df['보링회전 RPM-오른팔'] / (one_df['보링 가공 전,후 속도-오른팔'] + 1e-6)
# one_df['rpm_load_interaction_right'] = one_df['보링회전 RPM-오른팔'] * one_df['보링회전 부하율-오른팔']
# one_df['feed_load_ratio_right'] = one_df['보링 가공 전,후 속도-오른팔'] / (one_df['보링회전 부하율-오른팔'] + 1e-6)
# one_df['rpm_feed_diff_right'] = one_df['보링회전 RPM-오른팔'] - one_df['보링 가공 전,후 속도-오른팔']
# one_df['load_rpm_feed_interaction_right'] = one_df['보링회전 부하율-오른팔'] * one_df['보링회전 RPM-오른팔'] * one_df['보링 가공 전,후 속도-오른팔']
# one_df['cutting_energy_right'] = one_df['보링회전 RPM-오른팔'] * one_df['보링 가공 전,후 속도-오른팔'] * one_df['보링회전 부하율-오른팔']

### 일단 모델링

In [ ]:
df = pd.read_csv("./1호기.csv")

In [ ]:
from datetime import datetime

# 드릴 팁 교체 시 8분 이상동안 진행된 데이터만 가져오기 위해..
filtered_replacement_times = [
    (start, end) for start, end in timelines if (end - start).total_seconds() >= 480
]

# '등록일'을 datetime 타입으로 변환
df['등록일'] = pd.to_datetime(df['등록일'])

# 드릴 팁 교체 시간대의 데이터 필터링
replacement_data = []
for start, end in filtered_replacement_times:
    replacement_data.append(df[(df['등록일'] >= start) & (df['등록일'] <= end)])

# 병합하여 하나의 데이터프레임 생성
replacement_df = pd.concat(replacement_data, ignore_index=True)

In [ ]:
replacement_df

In [ ]:
# 드릴 팁 교체 전후 데이터 분리
pre_replacement_data = []
post_replacement_data = []

time_window = pd.Timedelta(minutes=5)  # 교체 전후 5분 비교

for start, end in filtered_replacement_times:
    pre_replacement_data.append(df[(df['등록일'] >= start - time_window) & (df['등록일'] < start)])
    post_replacement_data.append(df[(df['등록일'] > end) & (df['등록일'] <= end + time_window)])

# 병합하여 하나의 데이터프레임 생성
pre_replacement_df = pd.concat(pre_replacement_data, ignore_index=True)
post_replacement_df = pd.concat(post_replacement_data, ignore_index=True)

# 주요 변수 평균 비교
variables_to_compare = [
    '드릴 상,하 속도-왼팔', '드릴 상,하 부하율-왼팔', '드릴 상,하 속도-오른팔', '드릴 상,하 부하율-오른팔',
    '보링회전 RPM-왼팔', '보링회전 부하율-왼팔', '보링 가공 전,후 속도-왼팔', '보링 가공 전,후 부하율-왼팔',
    '싸이클타임-오른팔', '카운트-오른팔'
]

pre_avg = pre_replacement_df[variables_to_compare].mean()
post_avg = post_replacement_df[variables_to_compare].mean()

# 비교 데이터프레임 생성
comparison_df = pd.DataFrame({'교체 전 평균': pre_avg, '교체 후 평균': post_avg})

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mlp

mlp.rcParams['font.family'] = "Malgun Gothic"
mlp.rcParams['axes.unicode_minus'] = False

# 시각화 대상 변수
variables_to_plot = ['드릴 상,하 속도-왼팔', '드릴 상,하 부하율-왼팔', '드릴 상,하 속도-오른팔', '드릴 상,하 부하율-오른팔', '보링회전 RPM-왼팔']

# 교체 전/후 데이터에 '구분' 컬럼 추가
pre_replacement_df['구분'] = '교체 전'
post_replacement_df['구분'] = '교체 후'

# 병합하여 시각화할 데이터 생성
plot_data = pd.concat([pre_replacement_df, post_replacement_df])

# 시계열 그래프 생성
for var in variables_to_plot:
    plt.figure(figsize=(10, 5))
    for label, subset in plot_data.groupby('구분'):
        plt.plot(subset['등록일'], subset[var], label=label, alpha=0.7)
    plt.title(f'드릴 팁 교체 전후 {var} 변화')
    plt.xlabel('시간')
    plt.ylabel(var)
    plt.legend()
    plt.xticks(rotation=45)
    plt.show()


In [ ]:
# 드릴 팁 교체 시점에서 주요 변수들의 평균 및 분산 비교

# 교체 시점에서의 데이터 필터링 (교체 전후 5분 포함)
time_window = pd.Timedelta(minutes=5)

replacement_analysis_data = []
for start, end in filtered_replacement_times:
    replacement_analysis_data.append(df[(df['등록일'] >= start - time_window) & (df['등록일'] <= end + time_window)])

# 병합하여 하나의 데이터프레임 생성
replacement_analysis_df = pd.concat(replacement_analysis_data, ignore_index=True)

# 비교할 주요 변수
variables_for_analysis = [
    '드릴 상,하 속도-왼팔', '드릴 상,하 부하율-왼팔', '드릴 상,하 속도-오른팔', '드릴 상,하 부하율-오른팔',
    '보링회전 RPM-왼팔', '보링회전 부하율-왼팔', '싸이클타임-오른팔', '카운트-오른팔'
]

# 드릴 팁 교체 시점 데이터의 평균 및 표준편차 계산
replacement_stats = replacement_analysis_df[variables_for_analysis].agg(['mean', 'std']).T

# 전체 데이터와 비교
overall_stats = df[variables_for_analysis].agg(['mean', 'std']).T

# 비교 데이터프레임 생성
comparison_stats_df = pd.concat([overall_stats, replacement_stats], axis=1)
comparison_stats_df.columns = ['전체 평균', '전체 표준편차', '교체 시점 평균', '교체 시점 표준편차']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 교체 시점 데이터에 '교체 여부' 라벨 추가 (1: 교체 시점, 0: 정상 운행)
df['교체 여부'] = 0

for start, end in filtered_replacement_times:
    df.loc[(df['등록일'] >= start - time_window) & (df['등록일'] <= end + time_window), '교체 여부'] = 1

# 학습 데이터 준비 (특징 변수 및 라벨)
features = ['드릴 상,하 속도-왼팔', '드릴 상,하 부하율-왼팔', 
            '드릴 상,하 속도-오른팔', '드릴 상,하 부하율-오른팔', 
            '보링회전 RPM-왼팔', '보링회전 부하율-왼팔', 
            '싸이클타임-오른팔', '카운트-오른팔']

X = df[features]
y = df['교체 여부']

# 훈련/테스트 데이터 분할 (80% 훈련, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 모델 학습 (랜덤 포레스트 사용)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 모델 평가
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred, target_names=['정상 운행', '드릴 팁 교체'])

# 결과 출력
print(report)


In [ ]:
df = pd.read_csv("./1호기.csv")

In [ ]:
df_filter = (df['보링, 핀 이동구간-왼팔'] == 0) & (df['보링 가공 전,후 위치-왼팔'] == 90)
filtered_df = df.loc[~df_filter]


In [ ]:
target = [col for col in filtered_df.columns if "부하율" in col]
for col in target:
    filtered_df[f"{col} RS"] = filtered_df[col].rolling(window=300).sum()

In [ ]:
fig = go.Figure()
for col in filtered_df.columns:
    fig.add_trace(go.Scatter(
        x=filtered_df['등록일'], y=filtered_df[col], name=col
    ))
    
for time in timelines:
    fig.add_vrect(
        x0=time[0],
        x1=time[1],
        annotation_text="drill tip changed",
        fillcolor="green",
        opacity=0.3
    )
    
fig.show()

### 지금까지 시도한 내용들

1. drill wear rate equation을 활용한 Feature 생성
    - 결손되는 값들도 많고, 특이점 포착 X
2. 이동 평균, 합, 표준편차를 활용한 Feature 생성
    - 마찬가지로 특이점 포착은 불가했음
3. 부하량, RPM 위주의 변화율로 Feature로 생성
    - 마찬가지로 특이점 포착은 불가했음
4. BackGround Knowledge 활용, 해당 내용들에 초점을 맞춘 데이터분석

### 앞으로 시도해 볼 것들
1. 변화율의 이동합
2. 드릴 파손 데이터가 들어오기 시작하면 그때도..

In [ ]:
target_cols = [col for col in one_df.columns if "부하율" in col or "RPM" in col]
target_cols

In [ ]:
for diff_col in target_cols:
    one_df[f'{diff_col}_diff'] = one_df[diff_col].diff()

In [ ]:
target_cols2 = [col for col in one_df.columns if "diff" in col] + ['등록일']
target_df2 = one_df[target_cols2]

In [ ]:
for col in target_df2.select_dtypes(exclude="datetime64[ns]").columns:
    target_df2[f"{col}_mavg"] = target_df2[col].rolling(window=300).sum()

In [ ]:
timelines

In [ ]:
fig = go.Figure()
for col in target_df2.columns:
    fig.add_trace(go.Scatter(
        x=target_df2['등록일'], y=target_df2[col], name=col
    ))
    
for time in timelines:
    fig.add_vrect(
        x0=time[0],
        x1=time[1],
        fillcolor="green",
        opacity=0.3,
        annotation_text="drill tip changed"
    )

fig.show()

In [ ]:
one_df.drop([col for col in one_df.columns if "diff" in col], axis=1, inplace=True)

In [ ]:
one_df.info()